In [1]:
from audiolm_pytorch import SoundStream, SoundStreamTrainer
soundstream = SoundStream(
    codebook_size = 1024,
    rq_num_quantizers = 8,
)
soundstream.load("content/soundstream/results3/soundstream.9000.pt")
soundstream.to("cuda")



2023-01-05 14:37:34 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-01-05 14:37:34.600389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 14:37:35.098408: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-05 14:37:35.098456: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-05 14:37:35.098461: W tensorflow/compiler/tf

SoundStream(
  (encoder): Sequential(
    (0): CausalConv1d(
      (conv): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
    )
    (1): Sequential(
      (0): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (1): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(3,))
          )
          (1): ELU(alpha=1.0)
          (2): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(1,), stride=(1,))
          )
          (3): ELU(alpha=1.0)
        )
      )
      (2): Residual(
        (fn): Sequential(
          (0): CausalConv1d(
            (conv): Conv1d(32, 32, kernel_size=(7,), stride=(1,), dilation=(9,))
   

In [2]:
import sys
import torch
from importlib import reload
import network_models.soundstream_lstm.LSTM_dataset as lds
import torch.utils.data as data_utils
#from network_models.soundstream_lstm.LSTM_dataset  import AudioEmotionTessDataset
reload(lds)


tess_dataset = lds.AudioEmotionTessDataset("../tess")

encoded_dataset = lds.AudioEmotionTessSoundStreamEncodedDataset(dataSet= tess_dataset, soundStream=soundstream)

In [24]:
import importlib
from pathlib import Path
import torch
module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
if module_path not in sys.path:
    sys.path.append(module_path)
import network_models.soundstream_lstm.small_model as sm
importlib.reload(sm)

import network_models.soundstream_lstm.soundstream_datacollator as datacollator
data_collator = datacollator.SoundStreamDataCollator()

# print(encoded_dataset.__getitem__(3).shape)
model = sm.EmotionClassifierSevenEmos()
tensor = torch.tensor(encoded_dataset.__getitem__(3)[0])
tensor = torch.nn.functional.pad(tensor, (0, 200 - tensor.shape[1], 0, 0))
tensor = tensor.flatten()
print(tensor[0].shape)
model(torch.tensor(tensor))


torch.Size([])


/tmp/ipykernel_4332/3511952482.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model(torch.tensor(tensor))


EmotionClassifierSevenEmos(
  (linear1): Linear(in_features=102400, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=200, bias=True)
  (linear3): Linear(in_features=200, out_features=4, bias=True)
  (linear4): Linear(in_features=4, out_features=6, bias=True)
  (softmax): Softmax(dim=None)
)

In [11]:
import numpy as np
from network_models.soundstream_lstm.soundstream_datacollator import SoundstreamModelTrainer
from transformers import TrainingArguments, EvalPrediction

is_regression = False

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}


training_args = TrainingArguments(
    output_dir="content/models",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition"
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=5.0,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
)
trainer = SoundstreamModelTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)

512

In [9]:
len(encoded_dataset[0][1])

1

In [12]:
encoded_dataset[0]["encoded"]

TypeError: tuple indices must be integers or slices, not str